SECTION 1

LOAD CSV FILE

In [211]:
import pandas as pd
orders=pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


LOAD JSON FILE

In [212]:
import pandas as pd
users=pd.read_json('users.json')
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


LOAD SQL FILE

In [213]:
import sqlite3
import pandas as pd
conn=sqlite3.connect('restaurants.db')
file=open('restaurants.sql')
sql=file.read()

conn.executescript(sql)
restaurants=pd.read_sql('SELECT * FROM restaurants', conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


MERGE

In [214]:
data=pd.merge(orders, users, on='user_id', how='left') #left join

In [215]:
data=pd.merge(orders, users, on='user_id', how='left') #with order with users

In [216]:
final_data=pd.merge(data, restaurants, on='restaurant_id', how='left') #with rest

Create Final Dataset

In [217]:
final_data.to_csv('final_food_delivery_dataset.csv', index=False)

In [218]:
import pandas as pd
df=pd.read_csv('final_food_delivery_dataset.csv')
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


order trends

In [219]:
#orders overtime
df['order_date']=pd.to_datetime(df['order_date'])
df.groupby(df['order_date'].dt.month).size()

/tmp/ipython-input-2878964141.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date']=pd.to_datetime(df['order_date'])


,0
order_date,
1,831
2,785
3,903
4,812
5,844
6,784
7,859
8,851
9,812


In [220]:
#order peak time
df['order_date'].dt.hour.value_counts().sort_index()

,count
order_date,
0,10000


USER BEHAVIOR

In [221]:
df['user_id'].value_counts()

,count
user_id,
2973,13
1515,12
874,11
496,11
1337,11
...,...
1101,1
2074,1
1473,1


In [222]:
#CITY WISE
df['city'].value_counts()

,count
city,
Bangalore,2751
Chennai,2469
Pune,2430
Hyderabad,2350


RESTAURANT BEHAVIOR

In [223]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [224]:
df['cuisine'].value_counts()

,count
cuisine,
Mexican,2581
Italian,2532
Indian,2469
Chinese,2418


In [225]:
df.groupby('city')['order_id'].count()

,order_id
city,
Bangalore,2751
Chennai,2469
Hyderabad,2350
Pune,2430


In [226]:
#Spending: Gold vs Regular Users
df.groupby('membership')['total_amount'].sum()

,total_amount
membership,
Gold,3975364.89
Regular,4036259.23


In [227]:
df['total_amount'].sum() #Revenue Contribution

np.float64(8011624.12)

In [228]:
#Monthly Revenue Trend
df.groupby(df['order_date'].dt.month)['total_amount'].sum()

,total_amount
order_date,
1,663020.26
2,630867.40
3,716738.98
4,666097.18
5,668428.61
6,610822.93
7,688559.45
8,694987.58
9,653838.07


In [229]:
# Q1Which city has the highest total revenue (total_amount) from Gold members?
gold=df[df['membership']=='Gold']

In [230]:
city_revenue=gold.groupby('city')['total_amount'].sum()

In [231]:
city_revenue.sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [232]:
city_revenue.idxmax()

'Chennai'

In [233]:
# Q2Which cuisine has the highest average order value across all orders?

cuisine_avg = df.groupby('cuisine')['total_amount'].mean()
cuisine_avg.sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [234]:
cuisine_avg.idxmax()

'Mexican'

In [235]:
# Q3 How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_total=df.groupby('user_id')['total_amount'].sum()
high_value_users=user_total[user_total > 1000]

In [236]:
high_value_users.count()

np.int64(2544)

In [237]:
# Q4 Which restaurant rating range generated the highest total revenue?
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']
df['rating_range']=pd.cut(df['rating'], bins=bins, labels=labels)

In [238]:
rating_revenue=df.groupby('rating_range')['total_amount'].sum()
rating_revenue

/tmp/ipython-input-866473400.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue=df.groupby('rating_range')['total_amount'].sum()


,total_amount
rating_range,
3.0–3.5,1881754.57
3.6–4.0,1717494.41
4.1–4.5,1960326.26
4.6–5.0,2197030.75


In [239]:
rating_revenue.idxmax()

'4.6–5.0'

In [240]:
# Q5 Among Gold members, which city has the highest average order value?
gold = df[df['membership']=='Gold']

In [241]:
city_avg=gold.groupby('city')['total_amount'].mean()
city_avg.idxmax()

'Chennai'

In [242]:
# Q6 Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

restaurant_count=df.groupby('cuisine')['restaurant_id'].nunique()
restaurant_count


,restaurant_id
cuisine,
Chinese,120
Indian,126
Italian,126
Mexican,128


In [243]:
cuisine_revenue=df.groupby('cuisine')['total_amount'].sum()
cuisine_revenue

,total_amount
cuisine,
Chinese,1930504.65
Indian,1971412.58
Italian,2024203.80
Mexican,2085503.09


In [244]:
summary=pd.DataFrame({
    'restaurant_count': restaurant_count,
    'total_revenue': cuisine_revenue
})
summary

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [245]:
summary.sort_values(by=['restaurant_count', 'total_revenue'], ascending=[True, False])

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [246]:
# Q8 What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
total_orders=len(df)

In [247]:
gold_orders=len(df[df['membership']=='Gold'])

In [248]:
percentage=(gold_orders/total_orders)*100
round(percentage)

50

In [249]:
# Q9 Which restaurant has the highest average order value but less than 20 total orders?
order_count=df.groupby('restaurant_name_x')['order_id'].count()

In [250]:
avg_value=df.groupby('restaurant_name_x')['total_amount'].mean()

In [251]:
summary=pd.DataFrame({
    'order_count': order_count,
    'avg_order_value': avg_value
})

In [252]:
filtered=summary[summary['order_count']<20]

In [253]:
filtered.sort_values(by='avg_order_value', ascending=False)

,order_count,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [254]:
#Which combination contributes the highest revenue?
combo_revenue=df.groupby(['membership', 'cuisine'])['total_amount'].sum()
combo_revenue

membership  cuisine
Gold        Chinese     977713.74
            Indian      979312.31
            Italian    1005779.05
            Mexican    1012559.79
Regular     Chinese     952790.91
            Indian      992100.27
            Italian    1018424.75
            Mexican    1072943.30
Name: total_amount, dtype: float64

In [255]:
combo_revenue.sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [256]:
combo_revenue.idxmax()


('Regular', 'Mexican')

In [257]:
#During which quarter of the year is the total revenue highest?
df['order_date'] = pd.to_datetime(df['order_date'])

In [258]:
df['quarter'] = df['order_date'].dt.quarter

In [259]:
quarter_revenue = df.groupby('quarter')['total_amount'].sum()
quarter_revenue

,total_amount
quarter,
1,2010626.64
2,1945348.72
3,2037385.10
4,2018263.66


In [260]:
quarter_revenue.idxmax()

np.int32(3)

SECTION 2


In [261]:
# Q1How many total orders were placed by users with Gold membership?
df[df['membership']=='Gold'].shape[0]

4987

In [262]:
#Q2 What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
round(df[df['city']=='Hyderabad']['total_amount'].sum())


1889367

In [263]:
#Q3 How many distinct users placed at least one order?
df['user_id'].nunique()

2883

In [264]:
# Q4 What is the average order value (rounded to 2 decimals) for Gold members?
round(
    df[df['membership']=='Gold']['total_amount'].mean(),
    2
)


np.float64(797.15)

In [265]:
# Q5 How many orders were placed for restaurants with rating ≥ 4.5?
df[df['rating']>=4.5].shape[0]

3374

In [266]:
# Q6 How many orders were placed in the top revenue city among Gold members only?
top_city=(
    df[df['membership']=='Gold'].groupby('city')['total_amount'].sum().idxmax()
)
top_city


'Chennai'

In [267]:
df[
    (df['membership']=='Gold')&(df['city']==top_city)
].shape[0]

1337

SECTION 3 FIB

In [268]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range', 'quarter'],
      dtype='object')